In [ ]:
import numpy as np

import os

import torch
from torchvision.datasets import MNIST
from captum.attr import GradientShap, IntegratedGradients, Saliency

from torch.utils.data import DataLoader, RandomSampler, Subset
from torchvision import transforms

form
from lfxai.models.images import ClassifierMnist, EncoderMnist


In [ ]:
# Basic setup
torch.random.manual_seed(123)
batch_size = 128

# Model Args
image_height = 28
dim_latent = 4

In [ ]:
# Data loading
data_dir = "data/mnist"
shared_transform = transforms.Compose([transforms.ToTensor()])

train_dataset = MNIST(data_dir, train=True, download=True, transform=shared_transform
                                           )
test_dataset = MNIST(data_dir, train=False, download=True, transform=shared_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False
)

In [ ]:
#### Model loading ######
# Specification
dim_latent = 4
name = "TestClassifier"
classifier_state_dict_path = os.path.join('')

# Load
encoder = EncoderMnist(dim_latent)
classifier = ClassifierMnist(encoder, dim_latent, name)
classifier.load_state_dict(torch.load(classifier_state_dict_path), strict=True)

print("Classifier Loaded")

In [ ]:
# Instantiate GradShap For Encoder and Full Model
# Note this code is directly from Captum
gradshap_encoder = GradientShap(encoder)
gradshap_full_model = GradientShap(classifier)

